# DSS-Extensions — JSON exports

*Updated for DSS C-API version 0.13.2*

You can open and then run this notebook on Google Colab for a quick overview if you don't want to set up a local environment: **[Open in Colab](https://colab.research.google.com/github/dss-extensions/dss_python/blob/master/docs/examples/JSON.ipynb)**.

**Preparation**

To rerun this notebook:

- Install Python; we recommend using [miniforge from conda-forge](https://github.com/conda-forge/miniforge/#miniforge) -- after installation, either create a dedicated environment or make sure the basics are installed (`conda install jupyterlab pandas`)
- Install the DSS-Extensions (`python -m pip install dss-python[plot] opendssdirect.py`)
- Download the example files:
    - If you have Git installed (recommended), you can run the following cell.
    - Otherwise, download https://github.com/dss-extensions/electricdss-tst/archive/refs/heads/master.zip and extract the `electricdss-tst-master` folder in the same path as this notebook. Rename it to `electricdss-tst`.

In [1]:
! git clone --depth=1 -q https://github.com/dss-extensions/electricdss-tst

Cloning into 'electricdss-tst'...
remote: Enumerating objects: 1494, done.
remote: Counting objects: 100% (1494/1494), done.
remote: Compressing objects: 100% (1218/1218), done.
remote: Total 1494 (delta 354), reused 1236 (delta 250), pack-reused 0
Receiving objects: 100% (1494/1494), 50.59 MiB | 33.23 MiB/s, done.
Resolving deltas: 100% (354/354), done.


If you are running under Google Colab, running the following cell should install the packages.

In [2]:
import os, subprocess
if os.getenv("COLAB_RELEASE_TAG"):
    print(subprocess.check_output('pip install dss-python[plot] opendssdirect.py[extras] pandas==2', shell=True).decode())

## Introduction

*Although this is a Python notebook and uses DSS-Python, the same functions and behavior described here apply to all DSS-Extensions, even if they use different API layouts: OpenDSSDirect.py, OpenDSSDirect.jl, DSS Sharp, DSS MATLAB.*

Our alternative OpenDSS engine used in DSS-Extensions, implemented in the DSS C-API library, extends the API to provide some JSON export functions since version 0.12.0. Provided some constraints, most classes and properties could be exported as JSON, but a few properties didn't work correctly even with these contraints. **Warning: This feature is an API Extension and is not available in the official OpenDSS.**

Note that manually creating JSON through the classic DSS API (as implemented in the official COM DLL API) is possible but cumbersome, requiring manually tracking and converting strings to the types and several other details. We expect the DSS C-API implementation, being shared across all DSS-Extensions, can achieve better performance and remove this extra processing work.

We will use Pandas to show tables more easily in this document, but it is not required for exporting. In the near future, we plan to release integrated dataframe support for using with Pandas and similar workloads in Python and other languages.

For general interactive use of DSS objects, the work-in-progress [Obj and Batch APIs (working names)](https://dss-extensions.org/dss_python/obj.html) provides more direct manipulation of objects. Some initial examples are available on DSS-Python's tests: [test_obj.py](https://github.com/dss-extensions/dss_python/blob/master/tests/test_obj.py), [test_batch.py](https://github.com/dss-extensions/dss_python/blob/master/tests/test_batch.py). This new API will later be published as a separate module.

We invite users to provide feedback, including feature requests, to better guide future development. The Discussions or Issues features here are good places to give us feedback: https://github.com/dss-extensions/dss-extensions

*See also: [DSS-Extensions — OpenDSS: Overview of Python APIs](https://dss-extensions.org/python_apis.html)*

### How it works

The DSS engine maintains an order of each property of each object when they are created and populated. Our DSS engine walks through this list and export each of the properties, applying disambiguation and some other transforms. For exporting a whole class, the results for each object are collected in a JSON array.

Some redundant properties are skipped, exporting the primary value when possible instead. 

For classes related to transformers and `LineGeometry`, there are a few auxiliary properties in DSS (`wdg` and `cond`) that are used to iterate the substructures and fill/read the values. On the JSON export, this properties are skipped, and the values that usually refer to the substructure are represented as arrays, effectively returning the values for all substructures. For example, transformers have the `kVA` property, which in the DSS language and general property usage refer to the `kVA` of the active winding of a certain transformer. In our JSON export, `kVA` is an array of all `kVA` values of the windings. For this example, `kVA` is turned into the `kVAs` property, but some properties do not have array-style options:

- `Transformer` and `XfmrCode`: `MaxTap`, `MinTap`, `RdcOhms`, `NumTaps`, `Rneut`, `Xneut`
- `AutoTrans`: `MaxTap`, `MinTap`, `RdcOhms`, `NumTaps`
- `LineGeometry`: `x`, `h` (note that these are already arrays in `LineSpacing` in the DSS language)

All these are consequently exported as arrays (lists) in the JSON document.

Only native JSON types are used. Complex numbers are represented as pairs of floating-point numbers.

The redundant properties at marked `Redundant with` in [DSS-Extensions: OpenDSS Commands and Properties](https://dss-extensions.org/dss_properties.html).

See a special note about `LineGeometry` and `Line` at the end of this document.

### Future expectations 

Although the basics of the JSON export feature are done, we may still adjust some aspects for better user experience and uniformity. We do not want to distance the output too far from the official OpenDSS format, but introducing a few extra properties (especially for Lines and Transformers) may lessen some concerns of usability.

**Naming convention:** 

- As expected for the internal property names, the JSON exports and Obj/Batch APIs will be updated to match the new internal names. Only the case of the characters will change since that doesn't affect OpenDSS (OpenDSS is mostly case insensitive). For example `nconds` may be updated to `NConds` without negative effects. Since we already have an option to export lower-case names, users could use that if they do not want to worry about this specific change in the future.
- Since OpenDSS have some properties which are invalid as identifiers on the most popular programming languages, we may add an option to transform the related keys. For example, `%LoadLoss` could be replaced with `pctLoadLoss`, as already done to implement the Obj/Batch APIs in Python and C++. This is not a priority.

**Alternative layouts:** 

- For transformers and related objects, we may provide an option to output and input windings as a list of objects. For lines and related, conductor position objects could be used.
- For array outputs, when the arrays represent matrices, add an option to use array of arrays (`[[1,2], [3,4]]`) instead of plain arrays (`[1,2,3,4]`). 

**Export and import:** Currently, only exporting is supported due to the various ways some classes can be populated, optional fields/properties, and so on.  
There are plan to provide a more restricted alternative for importing from various formats to allow easier validation. If the data is uniform, roundtrip as dataframes (export JSON and reimporting it later as dataframes) through `batch_new(df=df)`, e.g. `dss.Obj.Line.batch_new(df=df_lines)`.

**GeoJSON:** GeoJSON export of the whole circuit, including buses, is expected in a future version. We didn't want to add too much complexity before confirming the output is correct and reasonable.

## Basic usage

In the classic API, the active element or a whole active class can be exported.

In [3]:
from dss import dss

dss.Text.Command = 'redirect electricdss-tst/Version8/Distrib/IEEETestCases/13Bus/IEEE13Nodeckt.dss'

In [4]:
# Select a load
dss.ActiveCircuit.Loads.Name = '670a'
dss.ActiveCircuit.ActiveDSSElement.Name

'Load.670a'

In [5]:
dss.ActiveCircuit.ActiveDSSElement.ToJSON()

'{"DSSClass":"Load","name":"670a","bus1":"670.1","phases":1,"conn":"wye","model":1,"kV":2.3999999999999999E+000,"kW":1.7000000000000000E+001,"kvar":1.0000000000000000E+001}'

For inspection or post-processing, we could load the JSON string in Python for better formatting:

In [6]:
import json
json.loads(dss.ActiveCircuit.ActiveDSSElement.ToJSON())

{'DSSClass': 'Load',
 'name': '670a',
 'bus1': '670.1',
 'phases': 1,
 'conn': 'wye',
 'model': 1,
 'kV': 2.4,
 'kW': 17.0,
 'kvar': 10.0}

In [7]:
dss.SetActiveClass('Load')
json.loads(dss.ActiveCircuit.ActiveClass.ToJSON())

[{'DSSClass': 'Load',
  'name': '671',
  'bus1': '671.1.2.3',
  'phases': 3,
  'conn': 'delta',
  'model': 1,
  'kV': 4.16,
  'kW': 1155.0,
  'kvar': 660.0},
 {'DSSClass': 'Load',
  'name': '634a',
  'bus1': '634.1',
  'phases': 1,
  'conn': 'wye',
  'model': 1,
  'kV': 0.277,
  'kW': 160.0,
  'kvar': 110.0},
 {'DSSClass': 'Load',
  'name': '634b',
  'bus1': '634.2',
  'phases': 1,
  'conn': 'wye',
  'model': 1,
  'kV': 0.277,
  'kW': 120.0,
  'kvar': 90.0},
 {'DSSClass': 'Load',
  'name': '634c',
  'bus1': '634.3',
  'phases': 1,
  'conn': 'wye',
  'model': 1,
  'kV': 0.277,
  'kW': 120.0,
  'kvar': 90.0},
 {'DSSClass': 'Load',
  'name': '645',
  'bus1': '645.2',
  'phases': 1,
  'conn': 'wye',
  'model': 1,
  'kV': 2.4,
  'kW': 170.0,
  'kvar': 125.0},
 {'DSSClass': 'Load',
  'name': '646',
  'bus1': '646.2.3',
  'phases': 1,
  'conn': 'delta',
  'model': 2,
  'kV': 4.16,
  'kW': 230.0,
  'kvar': 132.0},
 {'DSSClass': 'Load',
  'name': '692',
  'bus1': '692.3.1',
  'phases': 1,
  'co

Or we could load into a dataframe. Some classes fit dataframes well, some not so much. 

Users should be aware that the order of properties can change the final result for some classes. Transformers and lines use indexed structures that don't fit the tabular format by default.

In [8]:
import pandas as pd
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON())

,DSSClass,name,bus1,phases,conn,model,kV,kW,kvar
0,Load,671,671.1.2.3,3,delta,1,4.160,1155.0,660.0
1,Load,634a,634.1,1,wye,1,0.277,160.0,110.0
2,Load,634b,634.2,1,wye,1,0.277,120.0,90.0
3,Load,634c,634.3,1,wye,1,0.277,120.0,90.0
4,Load,645,645.2,1,wye,1,2.400,170.0,125.0
5,Load,646,646.2.3,1,delta,2,4.160,230.0,132.0
6,Load,692,692.3.1,1,delta,5,4.160,170.0,151.0
7,Load,675a,675.1,1,wye,1,2.400,485.0,190.0
8,Load,675b,675.2,1,wye,1,2.400,68.0,60.0
9,Load,675c,675.3,1,wye,1,2.400,290.0,212.0


In [9]:
dss.SetActiveClass('Transformer')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON())

,DSSClass,name,phases,windings,XHL,bus,conn,kV,kVA,%R,bank,%loadloss
0,Transformer,sub,3,2.0,0.008,"[sourcebus, 650]","[delta, wye]","[114.99999999999999, 4.16]","[5000.0, 5000.0]","[0.0005, 0.0005]",NaN,NaN
1,Transformer,reg1,1,NaN,0.010,"[650.1, rg60.1]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
2,Transformer,reg2,1,NaN,0.010,"[650.2, rg60.2]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
3,Transformer,reg3,1,NaN,0.010,"[650.3, rg60.3]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
4,Transformer,xfm1,3,2.0,2.000,"[633, 634]","[wye, wye]","[4.16, 0.4799999999999999]","[500.0, 500.0]","[0.55, 0.55]",NaN,NaN


This NaN values above should be the empty/non-filled properties. For example, check the JSON for `reg1` directly:

In [10]:
dss.ActiveCircuit.Transformers.Name = 'reg1'
json.loads(dss.ActiveCircuit.ActiveDSSElement.ToJSON())

{'DSSClass': 'Transformer',
 'name': 'reg1',
 'phases': 1,
 'bank': 'reg1',
 'XHL': 0.01,
 'kVA': [1666.0, 1666.0],
 'bus': ['650.1', 'rg60.1'],
 'kV': [2.4, 2.4],
 '%loadloss': 0.01}

For `reg1`, the properties `windings`, `conn` and `%R` were not provided. The order of the properties in this previous cell output are in the same order as the original DSS script.  
We can check how the properties were used in the input file for comparison:

In [11]:
with open('electricdss-tst/Version8/Distrib/IEEETestCases/13Bus/IEEE13Nodeckt.dss', 'r') as ckt_file:
    for text_line in ckt_file:
        if 'transformer.reg1' in text_line.lower():
            # Found the transformer
            print(text_line, end='')
            
            #...but need to check for the MORE command:
            text_line = next(ckt_file)
            while text_line.startswith('~'):
                print(text_line, end='')
                text_line = next(ckt_file)

            break

New Transformer.Reg1 phases=1 bank=reg1 XHL=0.01 kVAs=[1666 1666]
~ Buses=[650.1 RG60.1] kVs=[2.4  2.4] %LoadLoss=0.01


From this, note that `Buses` was reported in the `bus` field as an array, as well as `kVAs` and `kVs` were exported as a `kVA` and `kV` arrays, respectively. This is done by disambiguation process.

As a sidenote, the dataframe can be a little better with Pandas 2.0+, using NA instead of NaN (which forces converting integers to floats):

In [12]:
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(), dtype_backend='pyarrow')

,DSSClass,name,phases,windings,XHL,bus,conn,kV,kVA,%R,bank,%loadloss
0,Transformer,sub,3,2,0.008,"[sourcebus, 650]","[delta, wye]","[114.99999999999999, 4.16]","[5000.0, 5000.0]","[0.0005, 0.0005]",<NA>,<NA>
1,Transformer,reg1,1,<NA>,0.010,"[650.1, rg60.1]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
2,Transformer,reg2,1,<NA>,0.010,"[650.2, rg60.2]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
3,Transformer,reg3,1,<NA>,0.010,"[650.3, rg60.3]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
4,Transformer,xfm1,3,2,2.000,"[633, 634]","[wye, wye]","[4.16, 0.4799999999999999]","[500.0, 500.0]","[0.55, 0.55]",<NA>,<NA>


## Export option flags

Some flags are currently implemented to adjust the output:

- `Full`: Exports all properties, even if not filled by the user. This can be useful for debugging or just checking some default properties.
- `EnumAsInt`: Try to use integer representation for enums. Note that the integer values may be specific to a version of the engine — that is, don't use the integer values for long-term storage. The integer values may be useful to avoid the work of converting the strings to integer on user code and so on. The enums should be available in the DSS.Obj classes, and are exported from the `DSS_ExtractSchema` low-level function.
- `SkipRedundant`: Avoid outputting redundant properties.
- `FullNames`: Prefer listing object full names, including the class name, even where it is not required.
- `Pretty`: Pretty-format the output (as much as the default implementation of fpjson in Free Pascal allows). The output is a bit more readable without requiring loading the JSON.
- `ExcludeDisabled`: When exporting a whole class, skip disabled elements.
- `SkipDSSClass`: By default, a JSON field "DSSClass" is added with the name of the DSS class for each element. This may not always be required or useful, so we can skip it sometimes.
- `LowercaseKeys`: Use all lowercase keys instead of the internal variants for all keys. "DSSClass" is also converted to "dssclass".

These are bit flags, so you have to bitwise-or them together when multiple flags are used.

In [13]:
from dss import DSSJSONFlags

In [14]:
?? DSSJSONFlags

Init signature:
 DSSJSONFlags(
    value,
    names=None,
    *,
    module=None,
    qualname=None,
    type=None,
    start=1,
)
Docstring:      An enumeration.
Source:        
class DSSJSONFlags(IntFlag):
    Full = 0x00000001
    SkipRedundant = 0x00000002
    EnumAsInt = 0x00000004
    FullNames = 0x00000008
    Pretty = 0x00000010
    ExcludeDisabled = 0x00000020
    SkipDSSClass = 0x00000040
    LowercaseKeys = 0x00000080
File:           ~/projects/dss/dss_python/dss/enums.py
Type:           EnumMeta
Subclasses:     

Let's print the `Pretty`, `LowercaseKeys` and `Full` outputs with for `reg1`:

In [15]:
dss.ActiveCircuit.Transformers.Name = 'reg1'
print(dss.ActiveCircuit.ActiveDSSElement.ToJSON())

{"DSSClass":"Transformer","name":"reg1","phases":1,"bank":"reg1","XHL":1.0000000000000000E-002,"kVA":[1.6660000000000000E+003,1.6660000000000000E+003],"bus":["650.1","rg60.1"],"kV":[2.3999999999999999E+000,2.3999999999999999E+000],"%loadloss":1.0000000000000000E-002}


In [16]:
dss.ActiveCircuit.Transformers.Name = 'reg1'
print(dss.ActiveCircuit.ActiveDSSElement.ToJSON(DSSJSONFlags.Pretty))

{
  "DSSClass" : "Transformer",
  "name" : "reg1",
  "phases" : 1,
  "bank" : "reg1",
  "XHL" : 1.0000000000000000E-002,
  "kVA" : [
    1.6660000000000000E+003,
    1.6660000000000000E+003
  ],
  "bus" : [
    "650.1",
    "rg60.1"
  ],
  "kV" : [
    2.3999999999999999E+000,
    2.3999999999999999E+000
  ],
  "%loadloss" : 1.0000000000000000E-002
}


In [17]:
dss.ActiveCircuit.Transformers.Name = 'reg1'
print(dss.ActiveCircuit.ActiveDSSElement.ToJSON(DSSJSONFlags.LowercaseKeys))

{"dssclass":"Transformer","name":"reg1","phases":1,"bank":"reg1","xhl":1.0000000000000000E-002,"kva":[1.6660000000000000E+003,1.6660000000000000E+003],"bus":["650.1","rg60.1"],"kv":[2.3999999999999999E+000,2.3999999999999999E+000],"%loadloss":1.0000000000000000E-002}


In [18]:
dss.ActiveCircuit.Transformers.Name = 'reg1'
json.loads(dss.ActiveCircuit.ActiveDSSElement.ToJSON(DSSJSONFlags.Full))

{'DSSClass': 'Transformer',
 'name': 'reg1',
 'phases': 1,
 'windings': 2,
 'bus': ['650.1', 'rg60.1'],
 'conn': ['wye', 'wye'],
 'kV': [2.4, 2.4],
 'kVA': [1666.0, 1666.0],
 'tap': [1.0, 1.05625],
 '%R': [0.005, 0.005],
 'Rneut': [-1.0, -1.0],
 'Xneut': [0.0, 0.0],
 'buses': ['650.1', 'rg60.1'],
 'conns': ['wye', 'wye'],
 'kVs': [2.4, 2.4],
 'kVAs': [1666.0, 1666.0],
 'taps': [1.0, 1.05625],
 'XHL': 0.01,
 'XHT': 35.0,
 'XLT': 30.0,
 'Xscarray': [0.01],
 'thermal': 2.0,
 'n': 0.8,
 'm': 0.8,
 'flrise': 65.0,
 'hsrise': 15.0,
 '%loadloss': 0.01,
 '%noloadloss': 0.0,
 'normhkVA': 1832.6,
 'emerghkVA': 2499.0,
 'sub': False,
 'MaxTap': [1.1, 1.1],
 'MinTap': [0.9, 0.9],
 'NumTaps': [32, 32],
 'subname': '',
 '%imag': 0.0,
 'ppm_antifloat': 1.0,
 '%Rs': [0.005, 0.005],
 'bank': 'reg1',
 'XfmrCode': None,
 'XRConst': False,
 'X12': 0.01,
 'X13': 35.0,
 'X23': 30.0,
 'LeadLag': 'Lag',
 'WdgCurrents': '594.2557, (-28.695), 562.6086, (151.31), ',
 'Core': 'shell',
 'RdcOhms': [0.0001469387755

In [19]:
dss.SetActiveClass('Transformer')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(DSSJSONFlags.SkipDSSClass | DSSJSONFlags.EnumAsInt))

,name,phases,windings,XHL,bus,conn,kV,kVA,%R,bank,%loadloss
0,sub,3,2.0,0.008,"[sourcebus, 650]","[1, 0]","[114.99999999999999, 4.16]","[5000.0, 5000.0]","[0.0005, 0.0005]",NaN,NaN
1,reg1,1,NaN,0.010,"[650.1, rg60.1]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
2,reg2,1,NaN,0.010,"[650.2, rg60.2]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
3,reg3,1,NaN,0.010,"[650.3, rg60.3]",NaN,"[2.399999999999999, 2.399999999999999]","[1666.0, 1666.0]",NaN,reg1,0.01
4,xfm1,3,2.0,2.000,"[633, 634]","[0, 0]","[4.16, 0.4799999999999999]","[500.0, 500.0]","[0.55, 0.55]",NaN,NaN


In [20]:
dss.SetActiveClass('Line')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(DSSJSONFlags.SkipDSSClass))

,name,phases,bus1,bus2,linecode,Ratings,normamps,emergamps,length,units,Switch,r1,r0,x1,x0,C1,C0
0,650632,3,rg60.1.2.3,632.1.2.3,mtx601,[400.0],400.0,600.0,2000.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,632670,3,632.1.2.3,670.1.2.3,mtx601,[400.0],400.0,600.0,667.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,670671,3,670.1.2.3,671.1.2.3,mtx601,[400.0],400.0,600.0,1333.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,671680,3,671.1.2.3,680.1.2.3,mtx601,[400.0],400.0,600.0,1000.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,632633,3,632.1.2.3,633.1.2.3,mtx602,[400.0],400.0,600.0,500.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,632645,2,632.3.2,645.3.2,mtx603,[400.0],400.0,600.0,500.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,645646,2,645.3.2,646.3.2,mtx603,[400.0],400.0,600.0,300.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,692675,3,692.1.2.3,675.1.2.3,mtx606,[400.0],400.0,600.0,500.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,671684,2,671.1.3,684.1.3,mtx604,[400.0],400.0,600.0,300.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,684611,1,684.3,611.3,mtx605,[400.0],400.0,600.0,300.0,ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
dss.SetActiveClass('LineCode')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(DSSJSONFlags.SkipDSSClass)).head().T

,0,1,2,3,4
name,1,2,3,4,5
nphases,3,3,3,3,3
baseFreq,60.0,60.0,60.0,60.0,60.0
rmatrix,"[0.086666667, 0.029545455, 0.088371212, 0.0290...","[0.088371212, 0.02992424, 0.087405303, 0.02954...","[0.087405303, 0.029071969999999992, 0.08666666...","[0.087405303, 0.029924242000000004, 0.08837121...","[0.088371212, 0.029545455, 0.086666667, 0.0299..."
xmatrix,"[0.20416666700000002, 0.095018939, 0.198522727...","[0.19852272700000004, 0.080227273, 0.201723485...","[0.201723485, 0.07289772699999998, 0.204166667...","[0.201723485, 0.080227273, 0.19852272700000004...","[0.19852272700000004, 0.095018939, 0.204166667..."
cmatrix,"[2.851710071999999, -0.920293787, 3.004631862,...","[3.004631862, -0.5850112530000001, 2.71134756,...","[2.71134756, -0.350755566, 2.851710071999999, ...","[2.71134756, -0.5850112530000001, 3.004631862,...","[3.004631862, -0.920293787, 2.851710071999999,..."
units,NaN,NaN,NaN,NaN,NaN


In [22]:
dss.SetActiveClass('LineCode')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(DSSJSONFlags.SkipDSSClass | DSSJSONFlags.SkipRedundant | DSSJSONFlags.Full)).head().T

,0,1,2,3,4
name,1,2,3,4,5
nphases,3,3,3,3,3
r1,NaN,NaN,NaN,NaN,NaN
x1,NaN,NaN,NaN,NaN,NaN
r0,NaN,NaN,NaN,NaN,NaN
x0,NaN,NaN,NaN,NaN,NaN
C1,NaN,NaN,NaN,NaN,NaN
C0,NaN,NaN,NaN,NaN,NaN
units,none,none,none,none,none
rmatrix,"[0.086666667, 0.029545455, 0.088371212, 0.0290...","[0.088371212, 0.02992424, 0.087405303, 0.02954...","[0.087405303, 0.029071969999999992, 0.08666666...","[0.087405303, 0.029924242000000004, 0.08837121...","[0.088371212, 0.029545455, 0.086666667, 0.0299..."


## Using OpenDSSDirect.py

In [23]:
from opendssdirect import dss as odd

odd.Basic.SetActiveClass('Capacitor')
odd.ActiveClass.ToJSON()

'[{"DSSClass":"Capacitor","name":"cap1","bus1":"675","phases":3,"kvar":[6.0000000000000000E+002],"kv":4.1600000000000001E+000},{"DSSClass":"Capacitor","name":"cap2","bus1":"611.3","phases":1,"kvar":[1.0000000000000000E+002],"kv":2.3999999999999999E+000}]'

In [24]:
odd.Loads.First()
odd.Element.ToJSON()

'{"DSSClass":"Load","name":"671","bus1":"671.1.2.3","phases":3,"conn":"delta","model":1,"kV":4.1600000000000001E+000,"kW":1.1550000000000000E+003,"kvar":6.6000000000000000E+002}'

In [25]:
json.loads(odd.Element.ToJSON(DSSJSONFlags.Full))

{'DSSClass': 'Load',
 'name': '671',
 'phases': 3,
 'bus1': '671.1.2.3',
 'kV': 4.16,
 'kW': 1155.0,
 'pf': 0.8682431421244591,
 'model': 1,
 'yearly': None,
 'daily': None,
 'duty': None,
 'growth': None,
 'conn': 'delta',
 'kvar': 660.0,
 'Rneut': -1.0,
 'Xneut': 0.0,
 'status': 'Variable',
 'class': 1,
 'Vminpu': 0.95,
 'Vmaxpu': 1.05,
 'Vminnorm': 0.0,
 'Vminemerg': 0.0,
 'xfkVA': 0.0,
 'allocationfactor': 0.5,
 'kVA': 1330.2725284692608,
 '%mean': 50.0,
 '%stddev': 10.0,
 'CVRwatts': 1.0,
 'CVRvars': 2.0,
 'kwh': 0.0,
 'kwhdays': 30.0,
 'Cfactor': 4.0,
 'CVRcurve': None,
 'NumCust': 1,
 'ZIPV': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '%SeriesRL': 50.0,
 'RelWeight': 1.0,
 'Vlowpu': 0.5,
 'puXharm': 0.0,
 'XRharm': 6.0,
 'spectrum': 'defaultload',
 'basefreq': 60.0,
 'enabled': True,
 'like': ''}

## Using the Obj and Batch APIs

Both simple objects and batch collections have the `to_json()` method. The same option flags are accepted.

In [26]:
Load = dss.Obj.Load
Load[1].to_json()

'{"DSSClass":"Load","name":"671","bus1":"671.1.2.3","phases":3,"conn":"delta","model":1,"kV":4.1600000000000001E+000,"kW":1.1550000000000000E+003,"kvar":6.6000000000000000E+002}'

For the following cell, a batch of loads that use delta connections are selected and then exported to JSON.

In [27]:
from dss.IObj import Connection
print(Load.batch(conn=Connection.delta).to_json(DSSJSONFlags.Pretty))

[
  {
    "DSSClass" : "Load",
    "name" : "671",
    "bus1" : "671.1.2.3",
    "phases" : 3,
    "conn" : "delta",
    "model" : 1,
    "kV" : 4.1600000000000001E+000,
    "kW" : 1.1550000000000000E+003,
    "kvar" : 6.6000000000000000E+002
  },
  {
    "DSSClass" : "Load",
    "name" : "646",
    "bus1" : "646.2.3",
    "phases" : 1,
    "conn" : "delta",
    "model" : 2,
    "kV" : 4.1600000000000001E+000,
    "kW" : 2.3000000000000000E+002,
    "kvar" : 1.3200000000000000E+002
  },
  {
    "DSSClass" : "Load",
    "name" : "692",
    "bus1" : "692.3.1",
    "phases" : 1,
    "conn" : "delta",
    "model" : 5,
    "kV" : 4.1600000000000001E+000,
    "kW" : 1.7000000000000000E+002,
    "kvar" : 1.5100000000000000E+002
  }
]


## `Line` and `LineGeometry`: special cases for conductors

`LineGeometry` objects present some difficulties even in the official OpenDSS in DSS scripts, when using `save circuit`.  
To workaround it, we expose all conductors in the `wire` property, and leave `wires` as the original `wires` array. That is, we can interpret `wire` in the JSON output as the full array of **conductors**. The class names are included, so there is no ambiguity of what it actually contains.

For `Line`, `cncables` and `tscables` are suppressed, and we use `wires` for all data instead.

In [28]:
dss.AllowEditor = False
dss.Text.Command = 'redirect ./electricdss-tst/Test/IEEE13_Assets.dss'

In [29]:
dss.SetActiveClass('WireData')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON())

,DSSClass,name,normamps,diam,GMRac,Rdc,Runits,radunits,GMRunits,Rac
0,WireData,acsr_556_5,730.0,0.927,0.37320,0.035227,kft,in,in,NaN
1,WireData,acsr_4/0,340.0,0.563,0.09768,0.112121,kft,in,in,NaN
2,WireData,acsr_1/0,230.0,0.398,0.05352,0.212121,kft,in,in,NaN
3,WireData,cu_1/0,100.0,0.368,0.13356,NaN,mi,in,in,0.607


In [30]:
dss.SetActiveClass('LineSpacing')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON())

,DSSClass,name,nconds,nphases,units,x,h
0,LineSpacing,500,4,3,ft,"[-4.0, -1.0, 3.0, 0.0]","[28.0, 28.0, 28.0, 24.0]"
1,LineSpacing,505,3,2,ft,"[-4.0, 3.0, 0.0]","[28.0, 28.0, 24.0]"
2,LineSpacing,510,2,1,ft,"[0.5, 0.0]","[29.0, 24.0]"


In [31]:
dss.SetActiveClass('LineGeometry')
pd.set_option('display.max_colwidth', None)
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(), dtype_backend='pyarrow')

,DSSClass,name,nconds,nphases,reduce,spacing,x,h,units,wire
0,LineGeometry,601,4,3,True,500,NaN,NaN,<NA>,NaN
1,LineGeometry,602,4,3,True,500,NaN,NaN,<NA>,NaN
2,LineGeometry,603,3,2,True,505,NaN,NaN,<NA>,NaN
3,LineGeometry,604,3,2,True,505,NaN,NaN,<NA>,NaN
4,LineGeometry,605,2,1,True,510,NaN,NaN,<NA>,NaN
5,LineGeometry,606,3,3,True,<NA>,"[-0.5, 0.0, 0.5]","[-4.0, -4.0, -4.0]",ft,NaN
6,LineGeometry,607,2,1,True,<NA>,"[0.0, 0.25]","[-4.0, -4.0]",ft,"[TSData.ts_1/0, WireData.cu_1/0]"


Since some of the LineGeometry objects above use LineSpacing objects to fill `x` and `h`, we would need to inspect the data from the LineSpacing objects to get all the info.  
In this case, using `DSSJSONFlags.Full` can help. Notice in the next cell output that all LineGeometry have `x` and `h` filled.  
Manually inspecting the properties through the classic DSS property API or our new Obj API would also return the correct values.

In [32]:
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(DSSJSONFlags.Full | DSSJSONFlags.FullNames), dtype_backend='pyarrow')

,DSSClass,name,nconds,nphases,wire,x,h,units,normamps,emergamps,reduce,spacing,Seasons,Ratings,LineType,like
0,LineGeometry,601,4,3,"[WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_4/0]","[-4.0, -1.0, 3.0, 0.0]","[28.0, 28.0, 28.0, 24.0]",ft,730.0,1095.0,True,LineSpacing.500,1,[0.0],oh,
1,LineGeometry,602,4,3,"[WireData.acsr_4/0, WireData.acsr_4/0, WireData.acsr_4/0, WireData.acsr_4/0]","[-4.0, -1.0, 3.0, 0.0]","[28.0, 28.0, 28.0, 24.0]",ft,340.0,510.0,True,LineSpacing.500,1,[0.0],oh,
2,LineGeometry,603,3,2,"[WireData.acsr_1/0, WireData.acsr_1/0, WireData.acsr_1/0]","[-4.0, 3.0, 0.0]","[28.0, 28.0, 24.0]",ft,230.0,345.0,True,LineSpacing.505,1,[0.0],oh,
3,LineGeometry,604,3,2,"[WireData.acsr_1/0, WireData.acsr_1/0, WireData.acsr_1/0]","[-4.0, 3.0, 0.0]","[28.0, 28.0, 24.0]",ft,230.0,345.0,True,LineSpacing.505,1,[0.0],oh,
4,LineGeometry,605,2,1,"[WireData.acsr_1/0, WireData.acsr_1/0]","[0.5, 0.0]","[29.0, 24.0]",ft,230.0,345.0,True,LineSpacing.510,1,[0.0],oh,
5,LineGeometry,606,3,3,"[CNData.cn_250, CNData.cn_250, CNData.cn_250]","[-0.5, 0.0, 0.5]","[-4.0, -4.0, -4.0]",ft,260.0,390.0,True,<NA>,1,[0.0],oh,
6,LineGeometry,607,2,1,"[TSData.ts_1/0, WireData.cu_1/0]","[0.0, 0.25]","[-4.0, -4.0]",ft,165.0,247.5,True,<NA>,1,[0.0],oh,


In [33]:
dss.ActiveCircuit.Lines.Name = '684611'
json.loads(dss.ActiveCircuit.ActiveDSSElement.ToJSON())

{'DSSClass': 'Line',
 'name': '684611',
 'phases': 1,
 'bus1': '684.3',
 'bus2': '611.3',
 'spacing': '510',
 'wires': ['WireData.acsr_1/0', 'WireData.acsr_1/0'],
 'length': 300.0,
 'units': 'ft',
 'Ratings': [400.0],
 'normamps': 230.0,
 'emergamps': 345.0}

In [34]:
dss.Text.Command = 'redirect electricdss-tst/Test/IEEE13_LineAndCableSpacing.dss'

In [35]:
dss.SetActiveClass('Line')
pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(), dtype_backend='pyarrow').T

,0,1,2,3,4,5,6,7,8,9,10,11
DSSClass,Line,Line,Line,Line,Line,Line,Line,Line,Line,Line,Line,Line
name,650632,632670,670671,671680,632633,632645,645646,671684,684611,692675,684652,671692
bus1,rg60.1.2.3,632.1.2.3,670.1.2.3,671.1.2.3,632.1.2.3,632.3.2,645.3.2,671.1.3,684.3,692.1.2.3,684.1,671
bus2,632.1.2.3,670.1.2.3,671.1.2.3,680.1.2.3,633.1.2.3,645.3.2,646.3.2,684.1.3,611.3,675.1.2.3,652.1,692
spacing,500,500,500,500,500,505,505,505,510,515,520,<NA>
wires,"[WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_4/0]","[WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_4/0]","[WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_4/0]","[WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_556_5, WireData.acsr_4/0]","[WireData.acsr_4/0, WireData.acsr_4/0, WireData.acsr_4/0, WireData.acsr_4/0]","[WireData.acsr_1/0, WireData.acsr_1/0, WireData.acsr_1/0]","[WireData.acsr_1/0, WireData.acsr_1/0, WireData.acsr_1/0]","[WireData.acsr_1/0, WireData.acsr_1/0, WireData.acsr_1/0]","[WireData.acsr_1/0, WireData.acsr_1/0]",NaN,"[TSData.ts_1/0, WireData.cu_1/0]",NaN
length,2000,667,1333,1000,500,500,300,300,300,500,800,<NA>
units,ft,ft,ft,ft,ft,ft,ft,ft,ft,ft,ft,<NA>
Ratings,[400.0],[400.0],[400.0],[400.0],[400.0],[400.0],[400.0],[400.0],[400.0],[400.0],[400.0],NaN
normamps,-1,-1,-1,-1,-1,-1,-1,-1,-1,260,165,<NA>
